In [20]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

#langsmith tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACKING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [21]:

from langchain.document_loaders import WebBaseLoader
import bs4
webLoader = WebBaseLoader(web_paths=("https://docs.smith.langchain.com/observability/how_to_guides/tracing/trace_with_langchain",),bs_kwargs=dict(parse_only=bs4.SoupStrainer("div",attrs={"class": "theme-doc-markdown markdown"})))
scraped_documents = webLoader.load()

In [22]:
from langchain_text_splitters.character import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(scraped_documents)

In [23]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_API_KEY")

In [24]:
from langchain_openai import OpenAIEmbeddings

hf_embed = OpenAIEmbeddings()

In [25]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(split_documents, hf_embed)

In [26]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt_template = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:
<context>
{context}
</context>
""")


llm = ChatOpenAI(model="gpt-4o")
document_chain = create_stuff_documents_chain(llm, prompt_template)

In [27]:
input = "How is tracing in Langsmith done?"
faiss_retriever = db.as_retriever()

document_chain.invoke({
    "input": input,
    "context": faiss_retriever.invoke(input)
})

'How do you log a trace to LangSmith when using LangChain?\n\nNo extra code is needed to log a trace to LangSmith. You just run your LangChain code as you normally would.'

In [29]:
from langchain.chains.retrieval import create_retrieval_chain

retrieval_chain = create_retrieval_chain(faiss_retriever, document_chain)
response = retrieval_chain.invoke({"input": input})


In [32]:
print(response['answer'])

How can you seamlessly trace an application using LangChain (Python) and LangSmith SDK together?

You can seamlessly trace an entire application using LangChain (Python) and LangSmith SDK together by invoking LangChain objects within a traceable function. These objects will be traced and bound as a child run of the traceable function. No extra code is needed for this interoperability.


### Using Ollama Embeddings